In [ ]:
import sisl
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
%matplotlib inline

# TI-01 Spin Texture 
In this exercise, we learn how to calculate and plot spin-textures using ```sisl``` and ```SIESTA```, using buckled hexagonal bismuthene as an example.

## Exercise Overview
1. Create bismuthene geometry.
2. Generate ```SIESTA``` Hamiltonian.
3. Calculate the band structure.
4. Calculate the spin texture.

## Exercise

#### 1. Create the geometry in ``sisl`` and save it.
For this exercise, we model bismuthene in a buckled hexagonal phase. This crystal structure is similar to graphene, with two atoms per unit cell. However, the two sublattices form two parallel planes, separated by the buckling height $h$.
We use an in-plane lattice constant $a$ of $4.60\mathring{\textrm{A}}$ and buckling height $h$ of $1.62\mathring{\textrm{A}}$. The lattice constant for the orthogonal direction $c$ can be large, e.g. $40\mathring{\textrm{A}}$.

|Top view |Side View |
|:--------|:---------|
|<img src="img/Bi2D-BHex-c.png" alt="BHex-c" style="width: 400px;"/>|<img src="img/Bi2D-BHex-a.png" alt="BHex-a" style="width: 400px;"/>|

In [ ]:
# Define lattice vectors and atomic positions or use sisl.geom.honeycomb
# geom = sisl.Geometry(atomic_positions, lattice_vectors, atomic_species)
# geom = sisl.geom.honeycomb(bond, atoms)
geom.write('STRUC.fdf')

#### 2. Generate SIESTA Hamiltonian
A sample input file for ``SIESTA`` can be found in the *siesta_work* folder. We refer to the manual (/Docs/siesta.pdf) for the description of input parameters. The ``SIESTA`` Hamiltonian is required to compute the spin texture. ``SIESTA`` will store it if the flag ``CDF.Save``, ``SaveHS``, or ``TS.HS.Save`` is set to ```true``` in the input file.
```
    siesta Bi2D_BHex.fdf > Bi2D_BHex.out
```
We can check the output file to ensure that the calculation converged and no errors occurred.

_Notes:_
``SIESTA`` produces different output files depending on which flag was used to write the Hamiltonian. The ``SystemLabel.HSX`` file (create with ``SaveHS True``) does not contain all the information needed for the following steps. For this, we have to include in the work folder the files:
- ``SystemLabel.ORB_INDX`` for information on the Basis and auxiliary supercell,
- ``SystemLabel.EIG`` for the Fermi level,
- ``SystemLabel.XV`` or ``fdf``-file for the geometry.

#### 3 Bandstructure

We now use ```sisl``` to calculate the band structure along the $\mathrm{M}$-$\Gamma$-$\mathrm{K}$-$\mathrm{M}$ path, reading the ```SIESTA``` Hamiltonian (Bi2D_BHex.nc, Bi2D_BHex.HSX or Bi2D_BHex.TSHS). 

*Notes:*
- [seeK-path](https://www.materialscloud.org/work/tools/seekpath) can be used to find the k-point path in the Brillouin Zone.
- ```sisl``` can save the geometry in different file formats, the ```xsf``` format can be read by seeK-path
- seeK-path uses a standardized unit cell. The displayed k-points refer to the reciprocal cell corresponding to this standardized cell, not to the original one.

In [ ]:
# Read the Hamiltonian from siesta output and create sisl.BandStructure
# sile = sisl.get_sile('path/to/*.fdf')
# H = ...
# EFermi = ...
# kpath = sisl.BandStructure(...)

In [ ]:
# Calculate the bands
# bands = ...

In [ ]:
# Plot the band structure
# Feel free to re-use code from previous exercises

#### 4 Spin Texture

The spin moment of a state $\psi$ is given by the expectation value of the angular momentum operator $\langle\psi|\vec{\boldsymbol{\mathcal{S}}}|\psi\rangle$. It is a three-dimensional vector and we can express its components in terms of the Pauli matrices $\sigma_x$, $\sigma_y$, $\sigma_z$, and the overlap matrix $\mathbf{S}$ as

\begin{align}
    \boldsymbol{\mathcal S}_x &= \langle\psi| \sigma_x \mathbf{S} | \psi\rangle\\
    \boldsymbol{\mathcal S}_y &= \langle\psi| \sigma_y \mathbf{S} | \psi\rangle\\
    \boldsymbol{\mathcal S}_z &= \langle\psi| \sigma_z \mathbf{S} | \psi\rangle.
\end{align}

In a periodic system, we define the band structure as the dispersion of the eigenenergies in reciprocal space. Analogously, we define the spin texture as the dispersion of the spin moments of the eigenstates.

To calculate the spin texture along a path in reciprocal space we need to:

1. Calculate the eigenstates for a k-point

   We use the ``eigenstate`` routine of our  ``sisl.Hamiltonian`` to calculate all eigenstates at the given k-point. The routine returns an [``EigenstateElectron``](http://zerothi.github.io/sisl/docs/latest/api-generated/sisl.physics.EigenstateElectron.html) object, which contains all the eigenvectors and eigenvalues. This object also holds routines that enable the calculation of (projected) density of states, spin moments.
   ```
   H = sisl.Hamiltonian(...)
   eigs = H.eigenstate(k=[...])
   ```

2. Calculate the spin moments of the eigenstates

   The spin moments of the eigenstates can be calculated with the [``spin_moment``](http://zerothi.github.io/sisl/docs/latest/api-generated/sisl.physics.EigenstateElectron.html#sisl.physics.EigenstateElectron.spin_moment) routine.
   - All eigenstates at once:
   ```
   eigs.spin_moment()
   ```
   - A single state
   ```
   eigs[i].spin_moment()
   ```

3. Calculate the spin texture

   To calculate the full spin texture we loop over all k-points and repeatedly perform steps 1 and 2.
   ```
   kpath = sisl.BandStructure(H, ...)
   for ik, k in enumerate(kpath):
      # Perform steps 1 and 2
      ...
   ```
   *In exercise 2 we will explore alternative solutions to this problem.*

In [ ]:
# Calculate the spin moments for all eigenstates along the k-path.
# 

A convenient way to visualize the spin texture is by coloring the bands according to the spin moment. The template below can be used to plot the spin texture. 

For reference on how to create multi-colored lines refer to the matplotlib [documentation](https://matplotlib.org/3.1.1/gallery/lines_bars_and_markers/multicolored_line.html).

In [ ]:
def plot_spin_texture(kpath, bands, spin_moments):    
    lk = kpath.lineark()
    xtick, xtick_label = kpath.lineartick()    
    nk, nbands = bands.shape
    
    # Create a figure with three subplot one for each component of the spin moment
    fig, axes = plt.subplots(1, 3, figsize=(8, 4.5), dpi=300, sharex=True, sharey=True)

    # Set the range of z-values, which will determine the color.
    norm = plt.Normalize(-1, 1)

    # Iterate over the spin components
    for icomp, component in enumerate(['$S_x$', '$S_y$', '$S_z$']):
        # Iterate over all bands 
        for ibnd in range(nbands):
            # It is not possible to change the color of a line directly, so we create small 
            # line segements from one point on the x-axis to the next. These segments can 
            # then be colored individually.
            points = np.array([lk, bands[:, ibnd]]).T.reshape(-1, 1, 2)
            segments = np.concatenate([points[:-1], points[1:]], axis=1)
            # Create a collection of the segments and specify a map that assigns colors 
            # to the segments according to the z-value
            lc = LineCollection(segments, cmap='coolwarm', norm=norm)
            # Set the z-values 
            lc.set_array(spin_moments[:, ibnd, icomp])             
            lc.set_linewidth(3)

            # Add the LineCollection to the subplot
            line = axes[icomp].add_collection(lc)
            
        axes[icomp].set_title(component)

    # All subplots share the same axis settings, so we can just adjust them once     
    ymin, ymax = (-2, 2)
    axes[0].set_xlim(min(lk), max(lk))
    axes[0].set_ylim(ymin, ymax) 
    axes[0].set_ylabel('Eigenspectrum [eV]')
    axes[0].xaxis.set_ticks(xtick)
    axes[0].set_xticklabels(xtick_label)

    for axis in axes:
        for tick in xtick:
            axis.plot([tick, tick], [ymin, ymax], 'k', linewidth=0.5)
    
    # Add a colorbar to the plot
    fig.colorbar(line, ax=axes.ravel().tolist())
    plt.show()

In [ ]:
# Use the function above to plot the spin-texture
# plot_spin_texture(...)

## Exercise:
1. Note that every band is two-fold degenerate. How can we separately visualize the two degenerate bands?
   - What is the difference in spin textures of the two degenerate bands?
   - Why? 

   *Hint*: In which order are the bands are stored?

2. There is a very compact way to calculate the spin texture *and* band-structure eigenvalues in one call
   - when calling a method on a `sisl.BrillouinZone` object it allows for *more* keyword arguments, see [here](http://zerothi.github.io/sisl/docs/latest/api/physics.brillouinzone.html) under **Multiple quantities**
   - the computationally expensive part is calculating the eigenstates (values and vectors). So doing this
     once is preferred (especially for large structures). How can you wrap them in one command?

In [ ]:
#

In [ ]:
# The results obtained with the compact form can be compared to the onces above:
# np.allclose(spin_moments1, spin_moments2)

## Learned methods
- Calculating the eigenstates of a Hamiltonian
- Evaluating spin moments of eigenstates
- Computing the spin texture
- Visualizing spin textures as color in a band structure plot
- Calculating quantities along a path in k space efficiently